In [1]:
from kSZ_forecast_general_func import *
from kSZ_forecast_HOD import *
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [5]:
snap_info=Snap_Info()
for Snapshot in [1631, 2448, 3356, 5000]:
    print(snap_info.get_redshift(Snapshot))

[2.023]
[1.028]
[0.485]
[0.]


In [2]:
for Snapshot in [2746, 2181]:
    save_halo_np(Snapshot)

Reading /home/cossim/CosmicGrowth/6620/fof/gcatdbb02.6620.2746
Redshift =  [0.811]
The number of halo109049117
ng: 109049117 nmin: 10
Reading /home/cossim/CosmicGrowth/6620/fof/gcatdbb02.6620.2181
Redshift =  [1.272]
The number of halo108320064
ng: 108320064 nmin: 10


# halo mass correspond to number density

In [3]:
Snapshot = 2448

for Snapshot in [2746, 2181]: #1631, 2448
    print("\n", Snapshot)
    halox,halov, mh =ReadHalos_np(zstep=Snapshot)
    halo_mass = mh*pm
    lgM_h = np.log10(halo_mass)
    n_halo =np.array([    1,     2,     3,     5,    7,     9,    30,   70])*10**-4 #8
    N = (n_halo*1200**3).astype(np.int32)
    lgM_h = np.sort(lgM_h)[::-1]
    print([lgM_h[N]])



 2746
[array([13.29552559, 13.07578389, 12.93738013, 12.75388786, 12.61841748,
       12.51708179, 12.01958989, 11.64667789])]

 2181
[array([13.12538864, 12.92474522, 12.79866028, 12.62210969, 12.50280233,
       12.41234644, 11.95202925, 11.60092039])]


In [6]:
for i in range(8):
    print(n_halo[i], lgM_h[N[i]])

0.0001 13.216168839900718
0.0002 13.006133905672922
0.00030000000000000003 12.874050327841571
0.0005 12.698601801598041
0.0007 12.564708222336815
0.0009000000000000001 12.470152114722236
0.003 11.991070159237045
0.007 11.628949118591503


In [7]:
Snapshot = 1631
halox,halov, mh =ReadHalos_np(zstep=Snapshot)
halo_mass = mh*pm
lgM_h = np.log10(halo_mass)
n_halo =np.array([    1,     2,     3,     5,    7,     9,    30,   70])*10**-4 #8
N = (n_halo*1200**3).astype(np.int32)
lgM_h = np.sort(lgM_h)[::-1]
print(lgM_h[N])
for i in range(8):
    print(n_halo[i], lgM_h[N[i]])

[12.84230083 12.66537838 12.54430498 12.39486591 12.29210015 12.2125257
 11.80201392 11.48552698]
0.0001 12.842300829138871
0.0002 12.665378384218178
0.00030000000000000003 12.544304976842376
0.0005 12.394865912558135
0.0007 12.292100154901027
0.0009000000000000001 12.212525704225452
0.003 11.802013923016997
0.007 11.48552697628919


# Generate CMB mock

In [3]:
Grid=512
redshift = 2.0
print(cosmo.comoving_distance(z=redshift))
Thelen = Boxlen/h/cosmo.comoving_distance(z=redshift).value
print(Thelen*180/np.pi,cmbsur[0][0],cmbsur[0][1])
for i in range(100):
    break
    filename = "./CMB_maps/ACT_redshift_"+str(np.round(redshift, 1))+"_grid_"+str(Grid)+"_"+str(i).zfill(3)+".npy"
    print(filename)
    cmb_map_act = cmb2d(thelen=Thelen, grid = Grid, Delta_T=cmbsur[0][0],sigma=cmbsur[0][1])
    np.save(filename, cmb_map_act)
    

5254.070758579652 Mpc
18.431030272494475 11 1.26


# Generate galaxy mock

In [2]:

save_halo_np(1631)
save_halo_np(2448)

Reading /home/cossim/CosmicGrowth/6620/fof/gcatdbb02.6620.1631
Redshift =  [2.023]
The number of halo98269902
ng: 98269902 nmin: 10
Reading /home/cossim/CosmicGrowth/6620/fof/gcatdbb02.6620.2448
Redshift =  [1.028]
The number of halo109207047
ng: 109207047 nmin: 10


In [5]:
Snapshot = 2181
halox,halov, mh =ReadHalos_np(zstep=Snapshot)
halo_mass = mh*pm
lgM_h = np.log10(halo_mass)
#random_seed = 1
for random_seed in [100, 200, 300, 400]:
    np.random.seed(random_seed)
    desi_models = ["L0", "L1", "L2", "L3"]
    for i in range(4):
        hod_desi = HOD_ELG_DESI(model = desi_models[i])
        galaxy_cata = GalaxyCatalogueSnapshot(lgM_h = lgM_h)
        galaxy_cata.add_galaxies(hod_desi)
        N_cen = galaxy_cata.num_cen
        N_sat = galaxy_cata.num_sat
        print(N_cen, N_sat)


        filename = "halo_Temporary_storage/halos_"+str(Snapshot)+"_DESI_"+desi_models[i]+"_seed"+str(random_seed)+".npz"
        print(filename)
        np.savez(filename, N_cen = N_cen, N_sat = N_sat)
    np.random.seed(random_seed)
    hsc_models = ["NB912", "NB816"]
    for i in range(2):
        hod_hsc=HOD_ELG_HSC(model = hsc_models[i])
        galaxy_cata = GalaxyCatalogueSnapshot(lgM_h = lgM_h)
        galaxy_cata.add_galaxies(hod_hsc) #random
        N_cen = galaxy_cata.num_cen
        N_sat = galaxy_cata.num_sat
        print(N_cen, N_sat)

        filename = "halo_Temporary_storage/halos_"+str(Snapshot)+"_HSC_"+hsc_models[i]+"_seed"+str(random_seed)+".npz"
        print(filename)
        np.savez(filename, N_cen = N_cen, N_sat = N_sat)

[0 0 1 ... 0 0 0] [27 20 22 ...  0  0  0]
halo_Temporary_storage/halos_2181_DESI_L0_seed100.npz
[0 0 0 ... 0 0 0] [20 21 11 ...  0  0  0]
halo_Temporary_storage/halos_2181_DESI_L1_seed100.npz
[0 0 0 ... 0 0 0] [6 3 1 ... 0 0 0]
halo_Temporary_storage/halos_2181_DESI_L2_seed100.npz
[0 0 0 ... 0 0 0] [1 1 4 ... 0 0 0]
halo_Temporary_storage/halos_2181_DESI_L3_seed100.npz
[1 1 0 ... 0 0 0] [341 256 259 ...   0   0   0]
halo_Temporary_storage/halos_2181_HSC_NB912_seed100.npz
[1 1 1 ... 0 0 0] [168 130 155 ...   0   0   0]
halo_Temporary_storage/halos_2181_HSC_NB816_seed100.npz
[0 0 0 ... 0 0 0] [37 22 24 ...  0  0  0]
halo_Temporary_storage/halos_2181_DESI_L0_seed200.npz
[0 0 0 ... 0 0 0] [18 15 12 ...  0  0  0]
halo_Temporary_storage/halos_2181_DESI_L1_seed200.npz
[0 0 0 ... 0 0 0] [6 4 4 ... 0 0 0]
halo_Temporary_storage/halos_2181_DESI_L2_seed200.npz
[0 0 0 ... 0 0 0] [0 1 0 ... 0 0 0]
halo_Temporary_storage/halos_2181_DESI_L3_seed200.npz
[0 1 1 ... 0 0 0] [379 264 264 ...   0   0   0]
